# Dynamics of the system


In [3]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, vlatex
import numpy as np
from IPython.display import display, Math
from phaseportrait import PhasePortrait2D

# The equations


![alt text](assets/system_image.jpg)


# Variables & symbolic constants


Constants:

- $m_d, m_B, m_E, m_{BE}, m_{AD}$ -- masses of disk, connector, end-effector and rods $BE$ and $AD$ respectively
- $R, l_{BE}, l_{AD}$ -- radius of the disk, length of $BE$ and $AD$ respectively
- $I_e$ -- inertia of end-effector around center of mass
- Coordinates of point $A$
- Angle of rotation of gravity force $\theta_g$


In [41]:
m_d, m_B, m_E, m_BE, m_AD = sp.symbols("m_d, m_B, m_E, m_{BE}, m_{AD}")
m_rod = m_B + m_E + m_BE + m_AD
R, l_BE, l_AD = sp.symbols("R, l_{BE}, l_{AD}")
I_e = sp.symbols("I_e")
x_A, y_A = sp.symbols("x_A, y_A")
r_A = sp.Matrix([x_A, y_A])
theta_g = sp.symbols("theta_g")
g = sp.symbols("g")

I_d = m_d * R**2 / 2

Variables:

- $\theta(t)$ -- generalized coordinate, angle of rotations of the disk
- $\tau_{ext}$ -- external torque, applied to the disk


In [42]:
t = sp.symbols("t")
theta, tau_ext = dynamicsymbols("theta, tau_{ext}")
dtheta = theta.diff(t)

# System's points as function of coordinate


There are many points & parameters in the system, which is important to derive from the system coordinates. These are:

- $r_B, r_M, r_F, r_E$ -- coordinates of points $B, M, F, E$ respectively
- Angle $\alpha$ between rod and horizon
- $l_{com}$ is a specific parameter, representing distance between point B and the center of mass of the inner rod
- $\frac{r_B - r_a}{||r_B - r_A||}$, represented as `r_AB_norm` is a normal vector, collinear with the rod


In [47]:
x_B, y_B = R * sp.cos(theta), R * sp.sin(theta)
r_B = sp.Matrix([x_B, y_B])

alpha = sp.atan2(y_B - y_A, x_B - x_A)
r_AB_norm = sp.Matrix([sp.cos(alpha), sp.sin(alpha)])

r_M = r_A + r_AB_norm * l_AD / 2
r_F = r_B + r_AB_norm * l_BE / 2
r_E = r_B + r_AB_norm * l_BE

l_com = (r_B - r_A).norm() - l_AD / 2

In [48]:
l_com.diff(t)

(((-re(x_A) + re(R*cos(theta(t))))*Derivative(re(R*cos(theta(t))), t) + (-im(x_A) + im(R*cos(theta(t))))*Derivative(im(R*cos(theta(t))), t))*Abs(R*cos(theta(t)) - x_A)*sign(R*cos(theta(t)) - x_A)/(R*cos(theta(t)) - x_A) + ((-re(y_A) + re(R*sin(theta(t))))*Derivative(re(R*sin(theta(t))), t) + (-im(y_A) + im(R*sin(theta(t))))*Derivative(im(R*sin(theta(t))), t))*Abs(R*sin(theta(t)) - y_A)*sign(R*sin(theta(t)) - y_A)/(R*sin(theta(t)) - y_A))/sqrt(Abs(R*sin(theta(t)) - y_A)**2 + Abs(R*cos(theta(t)) - x_A)**2)

It is also quite handy to have an angular velocity $\dot{\alpha}$ to calculate kinetic energy of the system. Moreover, the variable $K$ is introduced, such that: $$\dot{\alpha} = K_\alpha \dot{\theta}$$


In [5]:
dalpha = sp.diff(alpha, t)
K_alpha = sp.diff(alpha, theta)

display(Math(f"$\dot{{\\alpha}} = {vlatex(dalpha)} = K \dot{{\\theta}}$"))

<IPython.core.display.Math object>

# Energy of the system


# Velocity of rod's center of mass


One of the key component of the kinetic energy of the system is the velocity of center of mass.


The position itself could be expressed as follows: $$r_{com} = \frac{1}{m_{rod}} (m_{AD} r_M + m_B r_B + m_{BE} r_F + m_E r_E)$$


Despite the `sympy` is cabable of differentiating the velocity symbolically, it is useful to derive these equations manually: $$v_{com} = \frac{1}{m_{rod}}\left( (m_{AD} \frac{l_{AD}}{2} + m_{BE} \frac{l_{BE}}{2} + m_E l_{BE}) \begin{bmatrix} -\sin \alpha \ \cos \alpha \end{bmatrix} K_\alpha + (m_B + m_{BE} + m_E) \begin{bmatrix} -\sin \theta \ \cos \theta \end{bmatrix}\right) \dot{\theta} = D \dot{\theta}$$


In [6]:
r_com = (m_AD * r_M + m_B * r_B + m_BE * r_F + m_E * r_E) / m_rod
D = (
    (m_AD * l_AD / 2 + m_BE * l_BE / 2 + m_E * l_BE)
    * sp.Matrix([-sp.sin(alpha), sp.cos(alpha)])
    * K_alpha
    + (m_B + m_BE + m_E) * r_AB_norm.diff(theta)
) / m_rod

# Inertia of the rod


The intertia of the rod around rotational point $B$ is as follows: $$I_r = \frac{m_{AD}}{3} (\frac{l_{AD}^2}{4} + 3 l_{com}^2) + \frac{1}{3} m_{BE} l_{BE}^2 + I_E + m_E l_{BE}^2$$


In [7]:
I_r = (
    m_AD / 3 * (l_AD**2 / 4 + 3 * l_com**2)
    + m_BE * l_BE**2 / 3
    + I_e
    + m_E * l_BE**2
)

# Kinetic Energy


Using all calculations above, the kinetic energy could be formulated as follows: $$K = \frac{1}{2} m_{rod} v_{com}^T v_{com} + \frac{1}{2} I_d \dot{\theta}^2 + \frac{1}{2} I_r \dot{\alpha}^2 = \frac{1}{2} M \dot{\theta}^2,$$ where: $$M = m_{rod} D^T D + I_d + I_r K^2_\alpha$$


In [8]:
M = m_rod * D.norm() ** 2 + I_d + I_r * K_alpha**2
M

R**2*m_d/2 + (-R*(-R*sin(theta(t)) + y_A)*sin(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) + R*(R*cos(theta(t)) - x_A)*cos(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))**2*(I_e + l_{BE}**2*m_E + l_{BE}**2*m_{BE}/3 + m_{AD}*(l_{AD}**2/4 + 3*(-l_{AD}/2 + sqrt(Abs(R*sin(theta(t)) - y_A)**2 + Abs(R*cos(theta(t)) - x_A)**2))**2)/3) + (Abs((-(R*sin(theta(t)) - y_A)*(R*(-R*sin(theta(t)) + y_A)*sin(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) - R*(R*cos(theta(t)) - x_A)*cos(theta(t))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))*(l_{AD}*m_{AD}/2 + l_{BE}*m_E + l_{BE}*m_{BE}/2)/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) + (R*sin(theta(t))/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2) - (R*cos(theta(t)) - x_A)*(-R*(R*sin(theta(t)) - y_A)*cos(theta(t)) + R*(R*cos(theta(t)) - x_A)*sin(theta(t)))/((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)**(3/2))*(m_B + m_E + m_{BE}))/

# Potential energy


The only source of potential energy in the system is a gravitational force, affecting . Hence, the equation for a potential energy is: $$P = m_{rod} g y_{com} \cos \theta_g$$


In [9]:
P = m_rod * g * r_com[1] * sp.cos(theta_g)
P

g*(R*m_B*sin(theta(t)) + m_E*(R*sin(theta(t)) + l_{BE}*(R*sin(theta(t)) - y_A)/sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)) + m_{AD}*(l_{AD}*(R*sin(theta(t)) - y_A)/(2*sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2)) + y_A) + m_{BE}*(R*sin(theta(t)) + l_{BE}*(R*sin(theta(t)) - y_A)/(2*sqrt((R*sin(theta(t)) - y_A)**2 + (R*cos(theta(t)) - x_A)**2))))*cos(theta_g)

# Resulting equation


Given values of $M$, $P$, and external torque law $\tau_{ext}$, one might formulate an equation: $$M \ddot{\theta} + \dot{M} \dot{\theta} - \frac{1}{2} \frac{d}{d\theta} (M\dot{\theta}^2) + \frac{dP}{d\theta} = \tau_{ext}$$

By expanding the term $\frac{1}{2} \frac{d}{d\theta} (M\dot{\theta}^2) = \frac{1}{2}\dot{M}\dot{\theta}$, obtain: $$M \ddot{\theta} + \frac{1}{2} \dot{M} \dot{\theta} + \frac{dP}{d\theta} = \tau_{ext}$$


In [10]:
M_dot = sp.diff(M, t)
dP_dtheta = sp.diff(P, theta)

In [11]:
ddtheta = (-M_dot * dtheta / 2 + dP_dtheta - tau_ext) / M